# 2. 从 URL 获取文字

下载所有 Earnings Call Transcript 文本，支持：
- 自动检测缺失年份并重新下载
- 只下载近10年（2015年及以后）的数据
- 使用 Chrome 9222 端口连接



In [1]:
# ========== 配置和导入 ==========

import os
import time
import glob
import json
import random
import re
from pathlib import Path
from datetime import datetime
from typing import Optional, Tuple

import asyncio
from playwright.async_api import async_playwright
from playwright.async_api._generated import Page

# 16 家公司列表（与 notebook 1 保持一致）
TICKER_LIST = [
    # Banks / Financial (14)
    "JPM",  # JPMorgan Chase
    "BK",   # Bank of New York Mellon
    "DAL",  # Delta Airlines
    "C",    # Citigroup
    "WFC",  # Wells Fargo
    "BAC",  # Bank of America
    "TSM",  # TSMC
    "MS",   # Morgan Stanley
    "GS",   # Goldman Sachs
    "BLK",  # BlackRock
    "MTB",  # M&T Bank
    "STT",  # State Street
    "PNC",  # PNC Financial
    "BR",   # Broadridge Financial

    # Tech / Mega Caps (add 16 → total 30)
    "AAPL",  # Apple
    "MSFT",  # Microsoft
    "AMZN",  # Amazon
    "NVDA",  # Nvidia
    "TSLA",  # Tesla
    "NFLX",  # Netflix

    "AMD",   # Advanced Micro Devices
    "AVGO",  # Broadcom
    "INTC",  # Intel
    "QCOM",  # Qualcomm

    "CRM",   # Salesforce
    "ADBE",  # Adobe
    "NOW",   # ServiceNow
    "SHOP",  # Shopify
]

# 只下载近10年的数据（2015年及以后）
MIN_YEAR = 2015

PROJECT_ROOT = Path("..").resolve()
JSON_DIR = PROJECT_ROOT / "data" / "transcript_index"
MAIN_OUTPUT_DIR = PROJECT_ROOT / "data" / "transcripts"

print(f"项目根目录: {PROJECT_ROOT}")
print(f"JSON索引目录: {JSON_DIR}")
print(f"Transcript保存目录: {MAIN_OUTPUT_DIR}")
print(f"目标公司数: {len(TICKER_LIST)}")
print(f"时间范围: {MIN_YEAR}年及以后")


项目根目录: /Users/xinyuewang/Desktop/1.27
JSON索引目录: /Users/xinyuewang/Desktop/1.27/data/transcript_index
Transcript保存目录: /Users/xinyuewang/Desktop/1.27/data/transcripts
目标公司数: 28
时间范围: 2015年及以后


In [2]:
# 3. 下载单个 URL 的 transcript（用于补全缺失的季度）
# 注意：此 cell 需要在运行 Cell 4 和 Cell 5（函数定义）之后才能运行

async def download_single_transcript(url: str, ticker: str, title: str = None):
    """
    下载单个 URL 的 transcript 到指定 ticker 目录
    与批量下载使用相同的处理逻辑
    
    Args:
        url: Seeking Alpha 的 transcript URL
        ticker: 公司代码（例如 "BK"）
        title: 可选，如果提供则使用此标题，否则从页面获取
    """
    print("=" * 60)
    print(f"下载单个 transcript: {ticker}")
    print(f"URL: {url}")
    print("=" * 60)
    
    ticker_output_dir = MAIN_OUTPUT_DIR / ticker.upper()
    ticker_output_dir.mkdir(parents=True, exist_ok=True)
    
    # 检查文件是否已存在（在连接浏览器之前）
    if title:
        safe_title = sanitize_filename(title)
        txt_path = ticker_output_dir / f"{safe_title}.txt"
        if txt_path.exists():
            print(f"⚠️ 文件已存在: {txt_path}")
            print("跳过下载。")
            return True
    
    # 连接浏览器并执行下载（所有操作都在 async with 块内）
    async with async_playwright() as p:
        try:
            browser = await p.chromium.connect_over_cdp("http://localhost:9222")
            print("✅ 成功连接到 Chrome")
            
            # 获取或创建 context
            if browser.contexts:
                context = browser.contexts[0]
            else:
                context = await browser.new_context()
            
            # 获取或创建 page
            if context.pages:
                page = context.pages[0]
                # 确保页面有效
                try:
                    await page.bring_to_front()
                except:
                    page = await context.new_page()
            else:
                page = await context.new_page()
            
            print("✅ 页面准备就绪")
            
            # 如果没有提供 title，从页面获取
            if not title:
                print("正在访问页面获取标题...")
                await page.goto(url, timeout=60000, wait_until="domcontentloaded")
                await wait_for_captcha_and_resume(page)
                await page.wait_for_load_state("networkidle", timeout=30000)
                
                page_title = await page.title()
                if "Earnings Call Transcript" in page_title:
                    title = page_title.split(" | ")[0] if " | " in page_title else page_title
                else:
                    title = url.split("/")[-1].replace("-", " ").title()
            
            safe_title = sanitize_filename(title)
            
            # 再次检查文件是否已存在
            txt_path = ticker_output_dir / f"{safe_title}.txt"
            if txt_path.exists():
                print(f"⚠️ 文件已存在: {txt_path}")
                print("跳过下载。")
                return True
            
            # 使用批量下载的相同函数下载 transcript
            print("开始下载 transcript...")
            result = await download_transcript(page, url, str(ticker_output_dir), safe_title)
            
            if isinstance(result, tuple):
                _, error_message = result
                print(f"❌ 下载失败: {error_message}")
                return False
            else:
                print(f"✅ 下载成功: {txt_path}")
                return True
                
        except Exception as e:
            print(f"❌ 连接或下载过程中出错: {e}")
            import traceback
            traceback.print_exc()
            return False




In [3]:
# 命名
def sanitize_filename(title):
    """
    Sanitizes a string to be a valid filename by replacing invalid characters.
    """
    return re.sub(r'[<>:"/\\|?*]', '_', title)


# 解决反爬验证机制
async def wait_for_captcha_and_resume(page: Page):
    """
    Checks for a captcha page and pauses the script for manual resolution.
    """
    print("🤖 Checking for CAPTCHA...")
    try:
        captcha_button = page.get_by_text("按住").first
        await captcha_button.wait_for(state="visible", timeout=5000)
        
        print("\n" + "="*50)
        print("⚠️ CAPTCHA detected! Please manually solve it in the browser.")
        print("💡 Once done, return to the console and press Enter to continue.")
        print("="*50 + "\n")
        
        await asyncio.to_thread(input, "Press Enter to continue...")
        
        await page.wait_for_load_state('domcontentloaded')
        print("✅ CAPTCHA solved, resuming execution.")
        
    except Exception:
        print("🚀 No CAPTCHA detected, continuing.")
        pass


In [4]:
# 辅助函数：提取年份和检查缺失年份

def extract_year_from_title(title: str) -> Optional[int]:
    """从标题中提取年份（例如 "Q1 2024 Earnings" -> 2024）"""
    match = re.search(r'\b(20\d{2})\b', title)
    return int(match.group(1)) if match else None


def check_missing_years(ticker: str, min_year: int = MIN_YEAR) -> Tuple[list, set]:
    """
    检查某个 ticker 已下载的文件中缺失哪些年份
    返回：(已下载的文件列表, 缺失的年份集合)
    """
    ticker_dir = MAIN_OUTPUT_DIR / ticker
    if not ticker_dir.exists():
        return [], set(range(min_year, datetime.now().year + 1))
    
    # 获取已下载的文件
    downloaded_files = list(ticker_dir.glob("*.txt"))
    years_downloaded = set()
    
    for file_path in downloaded_files:
        year = extract_year_from_title(file_path.stem)
        if year and year >= min_year:
            years_downloaded.add(year)
    
    # 期望的年份范围
    current_year = datetime.now().year
    expected_years = set(range(min_year, current_year + 1))
    missing_years = expected_years - years_downloaded
    
    return downloaded_files, missing_years

In [5]:
# 下载单页transcript
async def download_transcript(page: Page, url: str, output_dir: str, title: str):
    """
    Downloads a transcript using an existing Playwright page.
    Returns True if successful, a tuple (False, error_message) otherwise.
    """
    try:
        print(f"Navigating to: {url}")
        
        await page.goto(url, timeout=60000)
        
        await wait_for_captcha_and_resume(page)
        
        await page.wait_for_load_state("networkidle")

        # Use a more specific locator to avoid "strict mode" errors
        transcript_locator = 'div.T2G6W[data-test-id="content-container"]'
        
        error_page_check = page.locator('h1[data-test-id="yikes-page-title"]')
        if await error_page_check.is_visible():
            raise Exception("Page not found on Seeking Alpha.")
        
        await page.wait_for_selector(transcript_locator, timeout=10000)
        
        transcript_div = page.locator(transcript_locator)
        transcript_text = await transcript_div.inner_text()
        
        os.makedirs(output_dir, exist_ok=True)
        
        file_path = os.path.join(output_dir, f"{title}.txt")
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(transcript_text)
        
        print(f"✅ Transcript saved to {file_path}")
        return True

    except Exception as e:
        # Catch any exception and return it as a failure
        return False, str(e)

In [6]:
def extract_year_from_title(title: str) -> Optional[int]:
    """从标题中提取年份"""
    match = re.search(r'\b(20\d{2})\b', title)
    return int(match.group(1)) if match else None


def check_missing_years(ticker: str, min_year: int = MIN_YEAR) -> Tuple[list, set]:
    """
    检查某个 ticker 已下载的文件中缺失哪些年份
    返回：(需要下载的 URL 列表, 缺失的年份集合)
    """
    ticker_dir = MAIN_OUTPUT_DIR / ticker
    if not ticker_dir.exists():
        return [], set(range(min_year, datetime.now().year + 1))
    
    # 获取已下载的文件
    downloaded_files = list(ticker_dir.glob("*.txt"))
    years_downloaded = set()
    
    for file_path in downloaded_files:
        year = extract_year_from_title(file_path.stem)
        if year and year >= min_year:
            years_downloaded.add(year)
    
    # 期望的年份范围
    current_year = datetime.now().year
    expected_years = set(range(min_year, current_year + 1))
    missing_years = expected_years - years_downloaded
    
    return downloaded_files, missing_years


# 主流程
async def main(ticker_list, min_year: int = MIN_YEAR, check_missing: bool = True):
    """
    下载所有 ticker 的 transcript
    
    Args:
        ticker_list: 要处理的 ticker 列表
        min_year: 最小年份（只下载此年份及以后的数据）
        check_missing: 是否检查缺失年份并重新下载
    """
    print("=" * 60)
    print("开始下载 Earnings Call Transcript")
    print("=" * 60)
    print(f"目标公司: {ticker_list}")
    print(f"时间范围: {min_year}年及以后")
    print(f"Chrome 端口: 9222")
    print("\n请确保已运行以下命令启动 Chrome:")
    print("nohup /Applications/Google\\ Chrome.app/Contents/MacOS/Google\\ Chrome \\")
    print("  --remote-debugging-port=9222 \\")
    print("  --user-data-dir=/tmp/chrome_sa \\")
    print("  >/tmp/chrome_sa.log 2>&1 &")
    print("=" * 60)
    
    json_files = []
    for ticker in ticker_list:
        json_path = JSON_DIR / f"{ticker}.json"
        if json_path.exists() and json_path.stat().st_size > 0:
            json_files.append((ticker, json_path))
        else:
            print(f"⚠️ JSON not found for {ticker}, skipping.")

    async with async_playwright() as p:
        try:
            browser = await p.chromium.connect_over_cdp("http://localhost:9222")
            context = browser.contexts[0]
            page = context.pages[0] if context.pages else await context.new_page()
            print("✅ 成功连接到 Chrome")
        except Exception as e:
            print(f"❗ Failed to connect to browser instance: {e}")
            print("Please ensure you have run Chrome with --remote-debugging-port=9222")
            return

        for ticker, json_path in json_files:
            try:
                if json_path.stat().st_size == 0:
                    print(f"❗ Skipping empty file: {json_path.name}")
                    continue

                ticker_output_dir = MAIN_OUTPUT_DIR / ticker
                ticker_output_dir.mkdir(parents=True, exist_ok=True)
                
                # 检查缺失年份
                downloaded_files, missing_years = check_missing_years(ticker, min_year)
                print(f"\n{'='*60}")
                print(f"--- Processing Ticker: {ticker} ---")
                print(f"已下载文件数: {len(downloaded_files)}")
                if missing_years:
                    print(f"⚠️ 缺失年份: {sorted(missing_years)}")
                else:
                    print(f"✅ 所有年份（{min_year}年及以后）都已下载")
                
                with open(json_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                
                # 过滤：只处理指定年份及以后的数据
                results = []
                for item in data:
                    title = item.get('title', '')
                    link = item.get('link', '')
                    if not title or not link:
                        continue
                    
                    year = extract_year_from_title(title)
                    if year is None or year < min_year:
                        continue
                    
                    # 如果检查缺失年份，只下载缺失年份的数据
                    if check_missing and missing_years and year not in missing_years:
                        continue
                    
                    results.append((title, link))
                
                print(f"需要下载的 URL 数: {len(results)}")
                
                # 定义每个公司的失败日志文件路径
                failed_links_file = ticker_output_dir / "failed.json"
                failed_links = []
                
                # 获取已下载的文件名集合（用于快速检查）
                downloaded_filenames = {f.stem for f in downloaded_files}
                
                for title, link in results:
                    safe_title = sanitize_filename(title)
                    
                    # 检查是否已下载
                    if safe_title in downloaded_filenames:
                        print(f"✅ Transcript for '{title[:60]}...' already exists. Skipping.")
                        continue
                    
                    clean_url = link.split('#')[0]
                    if 'Earnings' in title:
                        download_result = await download_transcript(page, clean_url, str(ticker_output_dir), safe_title)
                        
                        # 检查下载结果，如果失败则添加到列表中
                        if isinstance(download_result, tuple):
                            _, error_message = download_result
                            failed_links.append({
                                "title": title,
                                "link": clean_url,
                                "error": error_message
                            })
                        
                        delay = random.uniform(2, 5)
                        print(f"Pausing for {delay:.2f} seconds...")
                        await asyncio.sleep(delay)
                
                # 循环结束后，将该公司的失败链接保存到其专属的 failed.json 文件
                if failed_links:
                    with open(failed_links_file, 'w', encoding='utf-8') as f:
                        json.dump(failed_links, f, indent=4, ensure_ascii=False)
                    print(f"❗ {len(failed_links)} failed links saved to {failed_links_file}")
                
                # 最终统计
                final_files = list(ticker_output_dir.glob("*.txt"))
                final_years = {}
                for f in final_files:
                    year = extract_year_from_title(f.stem)
                    if year:
                        final_years[year] = final_years.get(year, 0) + 1
                print(f"✅ {ticker} 完成！最终文件数: {len(final_files)}")
                print(f"   年份分布: {dict(sorted(final_years.items()))}")

            except json.JSONDecodeError:
                print(f"❗ Error: {json_path.name} is not a valid JSON file. Skipping.")
                continue
            except Exception as e:
                print(f"❗ An unexpected error occurred while processing {ticker}: {e}")
                import traceback
                traceback.print_exc()

    print("\n" + "="*60)
    print("--- All done. ---")
    print("="*60)

In [7]:
# 显示配置信息
print(f"JSON目录: {JSON_DIR}")
print(f"Transcript目录: {MAIN_OUTPUT_DIR}")
print(f"目标公司: {TICKER_LIST}")

JSON目录: /Users/xinyuewang/Desktop/1.27/data/transcript_index
Transcript目录: /Users/xinyuewang/Desktop/1.27/data/transcripts
目标公司: ['JPM', 'BK', 'DAL', 'C', 'WFC', 'BAC', 'TSM', 'MS', 'GS', 'BLK', 'MTB', 'STT', 'PNC', 'BR', 'AAPL', 'MSFT', 'AMZN', 'NVDA', 'TSLA', 'NFLX', 'AMD', 'AVGO', 'INTC', 'QCOM', 'CRM', 'ADBE', 'NOW', 'SHOP']


In [8]:
# ========== 执行下载 ==========
# 运行此 cell 来下载所有 14 家公司的 transcript

# 正常完整跑一遍（按缺失年份补齐）
await main(TICKER_LIST, min_year=2015, check_missing=False)


开始下载 Earnings Call Transcript
目标公司: ['JPM', 'BK', 'DAL', 'C', 'WFC', 'BAC', 'TSM', 'MS', 'GS', 'BLK', 'MTB', 'STT', 'PNC', 'BR', 'AAPL', 'MSFT', 'AMZN', 'NVDA', 'TSLA', 'NFLX', 'AMD', 'AVGO', 'INTC', 'QCOM', 'CRM', 'ADBE', 'NOW', 'SHOP']
时间范围: 2015年及以后
Chrome 端口: 9222

请确保已运行以下命令启动 Chrome:
nohup /Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome \
  --remote-debugging-port=9222 \
  --user-data-dir=/tmp/chrome_sa \
  >/tmp/chrome_sa.log 2>&1 &
❗ Failed to connect to browser instance: BrowserType.connect_over_cdp: connect ECONNREFUSED 127.0.0.1:9222
Call log:
  - <ws preparing> retrieving websocket url from http://localhost:9222

Please ensure you have run Chrome with --remote-debugging-port=9222


In [9]:
# ========== 可选工具：清理已经成功下载对应文件的 failed.json ==========

import json


def clean_failed_files(tickers=None):
    """根据当前已经存在的 .txt 文件，清理各 ticker 目录下的 failed.json。
    - 如果某条 failed 记录对应的 txt 已经下载成功，则从 failed.json 中删掉它
    - 如果删完之后列表为空，则直接删除整个 failed.json 文件
    """
    if tickers is None:
        tickers = TICKER_LIST

    for ticker in tickers:
        ticker_dir = MAIN_OUTPUT_DIR / ticker
        failed_path = ticker_dir / "failed.json"
        if not failed_path.exists():
            continue

        try:
            with open(failed_path, "r", encoding="utf-8") as f:
                failed_links = json.load(f)
        except Exception as e:
            print(f"⚠️ {ticker}: failed.json 读取失败，跳过。错误: {e}")
            continue

        new_failed = []
        for item in failed_links:
            title = item.get("title", "")
            safe_title = sanitize_filename(title)
            txt_path = ticker_dir / f"{safe_title}.txt"
            # 只有还没成功下载对应 txt 的，才保留在 failed 里
            if not txt_path.exists():
                new_failed.append(item)

        if new_failed:
            with open(failed_path, "w", encoding="utf-8") as f:
                json.dump(new_failed, f, indent=4, ensure_ascii=False)
            print(f"❗ {ticker}: 仍有 {len(new_failed)} 条链接失败，已更新 failed.json")
        else:
            failed_path.unlink()
            print(f"✅ {ticker}: 所有之前失败的链接都已成功下载，删除 failed.json")


# 用法示例：在重新跑过某些 ticker 之后，清理它们的 failed.json：
clean_failed_files(["PNC","MTB", "JPM","GS", "C","BR","BLK","BK"])  # 按需替换 ticker 列表


In [10]:
# 如果某些公司有 failed.json，想单独重试它们，可以把下面的示例解注释：
# 比如只重试 MS 和 WFC：
failed_tickers = ["STT", "PNC","MTB", "JPM","GS", "C","BR","BLK","BK",]
await main(failed_tickers, min_year=MIN_YEAR, check_missing=False)  # 不按年份跳过，重新尝试所有链接

开始下载 Earnings Call Transcript
目标公司: ['STT', 'PNC', 'MTB', 'JPM', 'GS', 'C', 'BR', 'BLK', 'BK']
时间范围: 2015年及以后
Chrome 端口: 9222

请确保已运行以下命令启动 Chrome:
nohup /Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome \
  --remote-debugging-port=9222 \
  --user-data-dir=/tmp/chrome_sa \
  >/tmp/chrome_sa.log 2>&1 &
❗ Failed to connect to browser instance: BrowserType.connect_over_cdp: connect ECONNREFUSED 127.0.0.1:9222
Call log:
  - <ws preparing> retrieving websocket url from http://localhost:9222

Please ensure you have run Chrome with --remote-debugging-port=9222
